In [1]:
import numpy as np
import xml.etree.ElementTree as ET
import pickle
from scipy import misc
import glob
from PIL import Image
import tensorflow as tf
from six.moves import cPickle
import pandas as pd

/root/tensorflow/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/tensorflow/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
im_sz = 500    # height and width to which all images will be resized to
batch_size=16  # dataset will be divided into batches of this size

#CELEBA dataset has this file , which should be used to divide the dataset into 
#training - 0, 
#validation -1 
#testing - 2 

evalData = pd.read_csv('evalpartition.txt',sep=" ",header=None,skipinitialspace=True,names=['image_name','dataset_type'])
evalData.set_index('dataset_type',inplace=True)
trainData = evalData.loc[0].reset_index()['image_name']
valData = evalData.loc[1].reset_index()['image_name']
testData = evalData.loc[2].reset_index()['image_name']

In [ ]:
# to account to different lightning conditions, this mean RGB is subtracted from all the images
IMG_MEAN = np.array((116.66876762,104.00698793,122.67891434), dtype=np.float32)  

# Bounding boxes of the faces in dataset are loaded into bbdata
bbdata = pd.read_csv('list_bbox_celeba.txt',sep=" ",header=0,skiprows=1,skipinitialspace=True)
bbdata.set_index('image_id',inplace=True)

In [ ]:
#We resize a given image to a standard size, subtract mean from it. Calculate a corresponding output portrait image using bounding box data
def preprocess_data(sess,data,boundbox):
    b=np.zeros([data.shape[0],data.shape[1],1],dtype=int)
    for i in range(0,data.shape[0]):
        for j in range(0,data.shape[1]):
            if((j>boundbox.x_1) and (j<(boundbox.x_1+boundbox.width)) and (i>boundbox.y_1) and (i<(boundbox.y_1+boundbox.height))):
                b[i,j,0] = 15 
    img_tensor = tf.image.resize_image_with_crop_or_pad(data, im_sz, im_sz)
    labels_tensor = tf.image.resize_image_with_crop_or_pad(b, im_sz, im_sz)
            
    img,lbls = sess.run([img_tensor,labels_tensor])
    img[:,:,:] = img[:,:,:] - IMG_MEAN
    return img,lbls

In [ ]:
# preprocess images and store each batch of images in a file, do the same for training, validation and testing datasets
sess = tf.Session()
total_features = np.zeros((batch_size,im_sz,im_sz,3))
total_output = np.zeros((batch_size,im_sz,im_sz,1))

total_features_val = np.zeros((batch_size,im_sz,im_sz,3))
total_output_val = np.zeros((batch_size,im_sz,im_sz,3))

num_train_batches = int(len(trainData)/batch_size)
num_val_batches = int(len(valData)/batch_size)
num_test_batches = int(len(yesyData)/batch_size)

for i in range(0,num_train_batches+1):
    for j in range(0,batch_size):
        imgName = trainData[int(i*batch_size)+j]
        inputimg = misc.imread("data/"+imgName)
        bb=bbdata.loc[imgName]
        total_features[j],total_output[j] = preprocess_data(sess,inputimg,bb)
        print("val "+str(i)+" "+ str(j))
    pickle.dump((total_features, total_output), open("preprocessed_data/train_data/pre_processed_batch_"+str(i)+".p", 'wb'))
    
    
for i in range(0,num_val_batches+1):
    for j in range(0,batch_size):
        imgName = valData[int(i*batch_size)+j]
        inputimg = misc.imread("data/"+imgName)
        bb=bbdata.loc[imgName]
        total_features[j],total_output[j] = preprocess_data(sess,inputimg,bb)
        print("val "+str(i)+" "+ str(j))
    pickle.dump((total_features, total_output), open("preprocessed_data/val_data/pre_processed_batch_"+str(i)+".p", 'wb'))
    
for i in range(0,num_test_batches+1):
    for j in range(0,batch_size):
        imgName = testData[int(i*batch_size)+j]
        inputimg = misc.imread("data/"+imgName)
        bb=bbdata.loc[imgName]
        total_features[j],total_output[j] = preprocess_data(sess,inputimg,bb)
        print("test "+str(i)+" "+ str(j))
    pickle.dump((total_features, total_output), open("preprocessed_data/test_data/pre_processed_batch_"+str(i)+".p", 'wb'))
